In [1]:
from tensorflow import keras

# (train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
import numpy as np
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=10000)

# restore np.load for future normal usage
np.load = np_load_old
del(np_load_old)


word_index = keras.datasets.imdb.get_word_index()

print('preprocessing...')
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=256)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=256)

x_val = x_train[:10000]
y_val = y_train[:10000]

x_train = x_train[10000:]
y_train = y_train[10000:]

C:\Users\EAZETNG\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\EAZETNG\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\EAZETNG\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\EAZETNG\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarnin

preprocessing...


In [3]:
print('build model')
model = keras.Sequential()
model.add(keras.layers.Embedding(10000, 16))
model.add(keras.layers.LSTM(100))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print('train model')
model.fit(x_train,
          y_train,
          epochs=10,
          batch_size=512,
          validation_data=(x_val, y_val),
          verbose=1)

build model
train model
Train on 15000 samples, validate on 10000 samples
Epoch 1/10
15000/15000 [==============================] - 125s 8ms/sample - loss: 0.6894 - acc: 0.5514 - val_loss: 0.6869 - val_acc: 0.5739
Epoch 2/10
15000/15000 [==============================] - 137s 9ms/sample - loss: 0.6602 - acc: 0.7077 - val_loss: 0.6461 - val_acc: 0.7066
Epoch 3/10
15000/15000 [==============================] - 139s 9ms/sample - loss: 0.5336 - acc: 0.7759 - val_loss: 0.4393 - val_acc: 0.7893
Epoch 4/10
15000/15000 [==============================] - 132s 9ms/sample - loss: 0.3537 - acc: 0.8625 - val_loss: 0.3323 - val_acc: 0.8595
Epoch 5/10
15000/15000 [==============================] - 131s 9ms/sample - loss: 0.2341 - acc: 0.9077 - val_loss: 0.3035 - val_acc: 0.8761
Epoch 6/10
15000/15000 [==============================] - 135s 9ms/sample - loss: 0.1670 - acc: 0.9404 - val_loss: 0.3075 - val_acc: 0.8680
Epoch 7/10
15000/15000 [==============================] - 133s 9ms/sample - loss: 0.13

In [11]:
# import h5py
from keras.models import model_from_json
from keras.models import load_model

# serialize model to JSON
#  the keras model which is trained is defined as 'model' in this example
model_json = model.to_json()


with open("models/model.json", "w") as json_file:
    json_file.write(model_json)

# print('save trained model...')
# model.save('models/sentiment_keras.h5')
# # del(model)

# print('load model...')
# loaded_model = load_model('models/sentiment_keras.h5')

# serialize weights to HDF5
model.save_weights("models/model.h5")

FileNotFoundError: [Errno 2] No such file or directory: 'models/model.json'

In [ ]:
# load json and create model
json_file = open("models/model.json", "r")
load_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(load_model_json)
# load weights into new model
loaded_model.load_weights("models/model.h5")

In [10]:
print('evaluation')
evaluation = model.evaluate(x_test, y_test, batch_size=512)
print('Loss:', evaluation[0], 'Accuracy:', evaluation[1])

evaluation
25000/25000 [==============================] - 52s 2ms/sample - loss: 0.3088 - acc: 0.8711
Loss: 0.30884710318565367 Accuracy: 0.87108


In [20]:
sample = 'Basically there a family where little boy Jake thinks there a zombie in his closet his parents are fighting all the time This movie is slower than soap opera and suddenly Jake decides to become Rambo and kill the zombie OK first of all when you re going to make film you must Decide if its thriller or drama As drama the movie is watchable Parents are divorcing arguing like in real life And then we have Jake with his closet which totally ruins all the film expected to see BOOGEYMAN similar movie and instead watched drama with some meaningless thriller spots out of just for the well playing parents descent dialogs As for the shots with Jake just ignore them'
sample_label = 0
# convert input sentence to tokens based on word_index
inps = [word_index[word] for word in sample.split() if word in word_index]
# the sentence length should be the same as the input sentences
inps = keras.preprocessing.sequence.pad_sequences([inps], maxlen=256)
print('Accuracy:', model.evaluate(inps, [sample_label], batch_size=1)[1])
print('Sentiment score: {}'.format(model.predict(inps)[0][0]))

InvalidArgumentError: indices[0,207] = 36112 is not in [0, 10000)
	 [[{{node embedding_1_1/embedding_lookup}}]]